In [39]:
from sdv.demo import load_timeseries_demo
import pandas as pd

In [40]:
data = pd.read_csv('./AAPL Historical Data.csv')

In [41]:
data.head()

,Date,Price
0,11/01/2013,19.86
1,12/01/2013,20.04
2,01/01/2014,17.88
3,02/01/2014,18.79
4,03/01/2014,19.17


In [47]:
context_columns = ['Price']

In [52]:
sequence_index = 'Date'

In [53]:
from sdv.timeseries import PAR


In [54]:
model = PAR(
   context_columns=context_columns,
   sequence_index=sequence_index,
  )

In [55]:
model.fit(data)

TypeError: can only concatenate list (not "str") to list